In [1]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using DataStructures
using OrderedCollections
using BenchmarkTools

## Funzione da ottimizzare

In [2]:
function fragmentlines(model)
    V,EV = model
    # Creo indice spaziale
    Sigma = spaceindex(model)
    # calcolo parametri d'intersezione degli spigoli
    lineparams = linefragments(V,EV,Sigma)
    # initialization of local data structures
    vertdict = OrderedDict{Array{Float64,1},Array{Int,1}}()
    pairs = collect(zip(lineparams, [V[:,e] for e in EV]))
    vertdict = OrderedDict{Array{Float64,1},Int}()
    #Inizializzo nuovi V, EV per aggiungere i nuovi vertici/spigoli dello splitting
    W = Array[]
    EW = Array[]
    k = 0
    # Ricostruisco i nuovi punti generati dall'intersezione tra spigoli
    # tramite i parametri d'intersezione
    # Per ogni spigolo...
    for (params,linepoints) in pairs
        v1 = linepoints[:,1] #Isolo primo punto dello spigolo
        v2 = linepoints[:,2] #Isolo secondo punto dello spigolo
        # Calcolo un array contenente tutti i punti d'intersezione sullo spigolo (tanti quanti
        # sono i parametri d'intersez)			
        points = [ v1 + t*(v2 - v1) for t in params]   # !!!! loved !!
        #Creo un array che conterrà gli id dei punti d'intersezione trovati (verticispigolo)
        vs = zeros(Int64,1,length(points))
        PRECISION = 8
        # Per ogni punto d'intersezione trovato sullo spigolo....
        for (h,point) in enumerate(points)
            #Approssimo coordinate del punto(x,y) trovato di un epsilon 
            point = map(approxVal(PRECISION), point)
            #Se non ho mai visto prima il punto....
            if haskey(vertdict, point) == false
                k += 1 #Genero ID punto 
                vertdict[point] = k #Associo l'ID al punto
                push!(W, point) #Pusho il punto(x,y) nell'array W
            end
            vs[h] = vertdict[point] #Assegno l'id del punto trovato nell'array dei punti d'intersezione
        end
        [push!(EW, [vs[k], vs[k+1]]) for k=1:length(vs)-1]
    end
    #se ho N punti d'intersezione trovati, genero N-1 spigoli 
    #ESEMPIO: se vs=[34,35,36,37] vs[h=1]=34, vs[h=2]=35, vs[h=3]=36, vs[h=4]=37
    # allora andrò a creare le coppie [34,35],[35,36],[36,37] come 3 spigoli. Queste coppie le pusho in EW
    W,EW = hcat(W...),convert(Array{Array{Int64,1},1},EW)
    V,EV = congruence((W,EW))
    return V,EV
end

fragmentlines (generic function with 1 method)

## Dipendenze della funzione

In [5]:
function spaceindex(model::Lar.LAR)::Array{Array{Int,1},1}
    V,CV = model[1:2]
    # se il modello è in 3d o 2d (guardo le righe di V, in 3d V è una 3xN, in 2d V è una 2xN)
    dim = size(V,1)
    cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]
    #----------------------------------------------------------
    bboxes = [hcat(boundingbox(cell)...) for cell in cellpoints]
    xboxdict = coordintervals(1,bboxes)
    yboxdict = coordintervals(2,bboxes)
    # xs,ys are IntervalTree type
    xs = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in xboxdict
        xs[tuple(key...)] = boxset
    end
    ys = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in yboxdict
        ys[tuple(key...)] = boxset
    end
    xcovers = boxcovering(bboxes, 1, xs)
    ycovers = boxcovering(bboxes, 2, ys)
    covers = [intersect(pair...) for pair in zip(xcovers,ycovers)]

    if dim == 3
        zboxdict = coordintervals(3,bboxes)
        zs = IntervalTrees.IntervalMap{Float64, Array}()
        for (key,boxset) in zboxdict
            zs[tuple(key...)] = boxset
        end
        zcovers = boxcovering(bboxes, 3, zs)
        covers = [intersect(pair...) for pair in zip(zcovers,covers)]
    end
    # remove each cell from its cover
    for k=1:length(covers)
        covers[k] = setdiff(covers[k],[k])
    end
    return covers
end

function boundingbox(vertices::Lar.Points)
   minimum = mapslices(x->min(x...), vertices, dims=2)
   maximum = mapslices(x->max(x...), vertices, dims=2)
   return minimum, maximum
end

function coordintervals(coord,bboxes)
    boxdict = OrderedDict{Array{Float64,1},Array{Int64,1}}()
    for (h,box) in enumerate(bboxes)
        key = box[coord,:]
        if haskey(boxdict,key) == false
            boxdict[key] = [h]
        else
            push!(boxdict[key], h)
        end
    end
    return boxdict
end

function boxcovering(bboxes, index, tree)
    covers = [[] for k=1:length(bboxes)]
    for (i,boundingbox) in enumerate(bboxes)
        extent = bboxes[i][index,:]
        iterator = IntervalTrees.intersect(tree, tuple(extent...))
        for x in iterator
            append!(covers[i],x.value)
        end
    end
    return covers
end

function linefragments(V,EV,Sigma)
    m = length(Sigma) 
    sigma = map(sort,Sigma) 
    reducedsigma = sigma 
    params = Array{Float64,1}[[] for i=1:m]
    for h=1:m
        if sigma[h] ≠ []
            line1 = V[:,EV[h]]
            for k in sigma[h]
                line2 = V[:,EV[k]]
                out = intersection(line1,line2) 
                if out ≠ ()
                    α,β = out
                    if 0<=α<=1 && 0<=β<=1
                        push!(params[h], α)
                        push!(params[k], β)
                    end
                end
            end
        end
    end
    fragparams = []
    for line in params
        push!(line, 0.0, 1.0)
        line = sort(collect(Set(line)))
        push!(fragparams, line)
    end
    return fragparams
end

function intersection(line1,line2)
    x1,y1,x2,y2 = vcat(line1...)
    x3,y3,x4,y4 = vcat(line2...)

    det = (x4-x3)*(y1-y2)-(x1-x2)*(y4-y3)
    if det != 0.0
        a = 1/det
        b = [y1-y2 x2-x1; y3-y4 x4-x3]  # x1-x2 => x2-x1 bug in the source link !!
        c = [x1-x3; y1-y3]
        (β,α) = a * b * c
    else
        if (y1==y2) == (y3==y4) || (x1==x2) == (x3==x4) # segments collinear
             return nothing
        else
             # segments parallel: no intersection
             return nothing
        end
    end
    return α,β
end

function congruence(model)
    W,EW = model
    balltree = NearestNeighbors.BallTree(W)
    r = 0.0000000001
    near = Array{Any}(undef, size(W,2))
    for k=1:size(W,2)
        near[k] = NearestNeighbors.inrange(balltree, W[:,k], r, true)
    end
    near = map(sort,near) 
    for k=1:size(W,2)
        W[:,k] = W[:,near[k][1]]
    end
    pointidx = [ near[k][1] for k=1:size(W,2) ] 
    invidx = OrderedDict(zip(1:length(pointidx), pointidx))
    V = [W[:,k] for k=1:length(pointidx)]
    EV = []
    for e in (EW)
        newedge = [invidx[e[1]],invidx[e[2]]]
        if newedge[1] !== newedge[2]
            push!(EV,newedge)
        end
    end
    EV = [EV[h] for h=1:length(EV) if length(EV[h])==2]
    EV = convert(Lar.Cells, EV)
    return hcat(V...),EV
end

function approxVal(PRECISION)
    function approxVal0(value)
    out = round(value, digits=PRECISION)
    if out == -0.0
        out = 0.0
    end
    return out
    end
    return approxVal0
end

approxVal (generic function with 1 method)

## 0) Benchmark vecchia funzione 

In [6]:
V  =[0.0 2.0 2.0 -2.0 4.0 4.0 6.0 2.0 -1.0; 
      0.0 2.0 0.0  2.0 4.0 6.0 6.0 6.0  3.0]
EV = [[1, 2], [3, 4], [5, 6], [7, 6], [8, 9]]
@btime fragmentlines((V,EV))

  219.036 μs (1686 allocations: 91.91 KiB)


([0.0 0.66666667 … 2.0 -1.0; 0.0 0.66666667 … 6.0 3.0], [[1, 2], [2, 3], [4, 2], [2, 5], [6, 7], [8, 7], [9, 10]])

## 1) Controllo se la funzione è type unstable


In [7]:
@code_warntype fragmentlines((V,EV))

Variables
  #self#::Core.Compiler.Const(fragmentlines, false)
  model::Tuple{Array{Float64,2},Array{Array{Int64,1},1}}
  @_3::Int64
  #1::var"#1#4"
  @_5::Int64
  Sigma::Array{Array{Int64,1},1}
  lineparams::Array{Any,1}
  pairs::Any
  vertdict::Union{OrderedDict{Array{Float64,1},Array{Int64,1}}, OrderedDict{Array{Float64,1},Int64}}
  k::Int64
  @_11::Any
  W::Any
  EW@_13::Core.Box
  V@_14::Core.Box
  EV::Any
  params::Any
  linepoints::Any
  @_18::Any
  #2::var"#2#5"{_A,_B} where _B where _A
  #3::var"#3#6"{Array{Int64,2}}
  v1::Any
  v2::Any
  points::Any
  vs::Array{Int64,2}
  PRECISION::Int64
  @_26::Union{Nothing, Tuple{Tuple{Int64,Any},Tuple{Int64,Any}}}
  h::Int64
  point::Any
  @_29::Int64
  V@_30::Union{}
  EW@_31::Union{}
  EW@_32::Union{}
  V@_33::Union{}

Body::Tuple{Any,Any}
1 ──        Core.NewvarNode(:(#1))
│           Core.NewvarNode(:(@_5))
│           Core.NewvarNode(:(lineparams))
│           Core.NewvarNode(:(pairs))
│           Core.NewvarNode(:(vertdict))
│      

la funzione NON è type unstable in quanto ho nell'output la stringa:
    
    Body::Tuple{Any,Any}